![](2024-08-14-18-07-11.png)

# DETECCION FILTER
## Author: Diversa
## Last Update: 03/10/2024
## Proyect: Feminist Urban Sense
## Contact: hello@diversa.studio
---

# Detection Filter Notebook

## Descripción General

Este notebook está diseñado para procesar y filtrar datos relacionados con la detección de ciertos patrones en imágenes o señales. Proporciona un pipeline para cargar los datos, aplicar algoritmos de detección, y filtrar los resultados según criterios específicos.

### Contenido

El notebook está dividido en las siguientes secciones:

1. **Carga de Datos**: Se importan y preparan los datos necesarios para el análisis.
2. **Preprocesamiento**: Se aplican métodos de limpieza y normalización de datos.
3. **Algoritmo de Detección**: Se implementa el algoritmo para detectar patrones específicos en los datos.
4. **Filtrado de Resultados**: Se aplica un filtro a los resultados para obtener aquellos que cumplan ciertos criterios.
5. **Visualización**: Se generan visualizaciones para interpretar los resultados de manera clara y eficiente.
6. **Exportación de Resultados**: Los resultados filtrados se exportan a un archivo para su posterior análisis.

### Requisitos

Este notebook está desarrollado en Python y requiere las siguientes librerías:

- `pandas`

### Cómo Usarlo

1. Clonar o descargar el notebook.
2. Asegurarse de tener instaladas las dependencias usando `pip` o `conda`:
    ```bash
    pip install pandas 
    ```
3. Abrir el notebook en un entorno compatible como Jupyter o Colab.
4. Ejecutar las celdas secuencialmente para cargar los datos, preprocesar, aplicar la detección y visualizar los resultados.


In [2]:
import pandas as pd
import numpy as np

In [ ]:
# Cargar el archivo de limpieza con múltiples hojas
sheets_dict = pd.read_excel("your_path", sheet_name=None)

# Cargar la tabla de detecciones finales
tabla_2 = pd.read_excel("your_path")

# Verificar los nombres originales de las columnas
print("Nombres originales de columnas en tabla_2:", tabla_2.columns)

# No es necesario renombrar columnas porque ya tienen el formato correcto (con mayúscula 'Angle_0', etc.)

# Convertir las columnas a cadenas para evitar errores con .str
tabla_2['country'] = tabla_2['country'].astype(str).str.lower().str.strip()
tabla_2['numero_img'] = tabla_2['numero_img'].astype(str).str.lower().str.strip()
tabla_2['category'] = tabla_2['category'].astype(str).str.lower().str.strip()

# Lista de países que esperamos encontrar
paises = ['argentina', 'brasil', 'ec_co', 'peru', 'tailandia', 'vietnam']

# Conteo de datos no nulos antes de la modificación
conteo_inicial = tabla_2[['Angle_0', 'Angle_90', 'Angle_180', 'Angle_270']].notnull().sum()

# Iteramos por cada hoja del archivo (cada categoría)
for category, tabla_1 in sheets_dict.items():
    # Limpiar los nombres de las columnas y los valores
    tabla_1.columns = tabla_1.columns.str.lower().str.strip()
    
    # Limpieza de valores en tabla_1
    tabla_1 = tabla_1.apply(lambda x: x.str.strip().str.lower() if x.dtype == "object" else x)

    # Iteramos por cada país (columna) en la hoja actual
    for country in paises:
        if country in tabla_1.columns:
            # Filtramos solo las filas donde haya datos de imágenes para ese país
            country_data = tabla_1[[country]].dropna()

            # Iteramos sobre cada imagen en esa columna del país
            for _, row in country_data.iterrows():
                # Extraemos el código de la imagen (e.g., streetview_35_90_out)
                image_code = row[country]

                # Comprobar si image_code no está vacío y es una cadena
                if pd.notna(image_code) and isinstance(image_code, str):
                    parts = image_code.split('_')
                    # Validar que se obtuvieron al menos tres partes
                    if len(parts) >= 3:
                        numero_img = parts[1].strip()
                        angulo = parts[2].strip()  # Mantener como cadena
                        
                        # Verificar que el ángulo está en el formato correcto (0, 90, 180, 270)
                        if angulo in ['0', '90', '180', '270']:
                            column_name = f"Angle_{angulo}"  # Formar el nombre de la columna con la "A" mayúscula
                            
                            # Verificar si la columna existe antes de modificar
                            if column_name in tabla_2.columns:
                                # Comprobar si la fila existe antes de modificar
                                mask = (
                                    (tabla_2['country'] == country) &
                                    (tabla_2['numero_img'] == numero_img) &
                                    (tabla_2['category'] == category)
                                )
                                
                                # Solo modificar si existe la coincidencia
                                if mask.any():
                                    # Imprimir valor antes del cambio
                                    valor_antes = tabla_2.loc[mask, column_name].values[0]
                                    print(f"Valor antes del cambio: {valor_antes} para {country}, numero_img: {numero_img}, ángulo: {angulo}")
                                    
                                    # Realizar la modificación con np.nan
                                    tabla_2.loc[mask, column_name] = np.nan  # Reemplaza por NaN explícitamente
                                    
                                    # Imprimir valor después del cambio
                                    valor_despues = tabla_2.loc[mask, column_name].values[0]
                                    print(f"Valor después del cambio: {valor_despues} para {country}, numero_img: {numero_img}, ángulo: {angulo}")
                                    
                                    # Verificación de éxito
                                    if pd.isna(tabla_2.loc[mask, column_name]).all():
                                        print(f"Fila modificada correctamente para {country}, numero_img: {numero_img}, ángulo: {angulo}")
                                    else:
                                        print(f"Error: la fila no fue modificada para {country}, numero_img: {numero_img}, ángulo: {angulo}")
                                else:
                                    print(f"No se encontró coincidencia para {country}, numero_img: {numero_img}, ángulo: {angulo}")
                            else:
                                print(f"Columna '{column_name}' no encontrada.")
                        else:
                            print(f"Ángulo inesperado: '{angulo}' en el código de imagen: '{image_code}'")
                    else:
                        print(f"Formato inesperado en el código de imagen: '{image_code}'. Debe contener al menos 3 partes.")
                else:
                    print(f"image_code vacío o no es una cadena: '{image_code}'")

# Guardamos la tabla limpia
tabla_2.to_csv("tabla_limpia_f.csv", index=False)

# Conteo de datos no nulos después de la modificación
conteo_final = tabla_2[['Angle_0', 'Angle_90', 'Angle_180', 'Angle_270']].notnull().sum()
print("Conteo de datos no nulos después de la modificación:")
print(conteo_final)

# Mostrar las diferencias
diferencias = conteo_inicial - conteo_final
print("Diferencia de datos eliminados:")
print(diferencias)


In [31]:
df = pd.read_csv('your_path_here.csv')

# Obtener una lista de ciudades únicas
unique_cities = df['city'].unique()

# Imprimir las ciudades únicas
for city in unique_cities:
    print(city)


Rosario
JaraguÃ¡ do Sul
MedellÃ­n
Mandalay
Bangkok
HanÃ³i
Piura
Quito
Cuenca


In [ ]:
df = pd.read_csv('your_path_here.csv')

# Palabras a corregir (incorrectas a corregir)
correcciones = {
    'Rosario': 'rosario',
    'JaraguÃ¡ do Sul': 'jaragua_do_sul',
    'MedellÃ­n': 'medellin',
    'Mandalay': 'mandalay',
    'Bangkok': 'bangkok',
    'HanÃ³i': 'hanoi',
    'Piura': 'piura',
    'Quito': 'quito',
    'Cuenca': 'cuenca'
}

# Corregir las palabras en las columnas
def corregir_palabras(texto):
    # Reemplazar la palabra errónea por la correcta
    for palabra_erronea, palabra_correcta in correcciones.items():
        texto = texto.replace(palabra_erronea, palabra_correcta)
    return texto

# Aplicar la corrección en las columnas 'city' e 'indice'
df['city'] = df['city'].apply(corregir_palabras)
df['indice'] = df['indice'].apply(corregir_palabras)

# Mostrar el DataFrame corregido
print(df)

# Guardar el DataFrame corregido en un nuevo archivo CSV
df.to_csv('detecciones_finales.csv', index=False)
